ReportLab, Flux
==

L'idée de ce chapitre est de présenter les bases de reportlab et de montrer comment générer un document contenant un flux de données qui vont s'enchaîner de page en page.

In [ ]:
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.styles import ParagraphStyle
from reportlab.lib.enums import TA_JUSTIFY
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm

from reportlab.platypus import Paragraph
from reportlab.platypus import SimpleDocTemplate
from reportlab.platypus import Spacer
from reportlab.platypus import Image
from reportlab.platypus import PageBreak
from reportlab.platypus import Table

from reportlab.graphics.shapes import Drawing
from reportlab.graphics.charts.linecharts import HorizontalLineChart

Gestion des styles
--

In [ ]:
styles = getSampleStyleSheet()

In [ ]:
styles.list()

In [ ]:
styles.byName

In [ ]:
styles.byAlias

In [ ]:
styles.get("df")

In [ ]:
style = styles.get("BodyText")

In [ ]:
dir(style)

In [ ]:
mon_style = ParagraphStyle(name='mon_style', alignment=TA_JUSTIFY, fontName = "Helvetica", fontSize = 14)

In [ ]:
styles.add(mon_style)

In [ ]:
from copy import copy
normal_justifié = copy(style)
normal_justifié.name='Just'
normal_justifié.alignment=TA_JUSTIFY
normal_justifié.fontName = "Helvetica"
normal_justifié.fontSize = 14
styles.add(normal_justifié)

In [ ]:
style_tableau = [
    ('ALIGN',         (0,0),  (-1,-1), "LEFT"),
    ('VALIGN',        (0,0),  (-1,-1), "TOP"),
    ('LEFTPADDING',   (0,0),  (-1,-1), 0*cm),
    ('RIGHTPADDING',  (0,0),  (-1,-1), 0*cm),
    ('TOPPADDING',    (0,0),  (-1,-1), 0*cm),
    ('BOTTOMPADDING', (0,0),  (-1,-1), 0*cm),
]

In [ ]:
style_tableau1 = style_tableau[:]
style_tableau1.append(('LINEABOVE', (0,0), (-1, 0), 1, colors.turquoise))
style_tableau1.append(('LINEABOVE', (0,1), (-1,-1), 0.5, colors.darkturquoise))

Ecrire les données
--

In [ ]:
flowables = []

In [ ]:
flowables.append(Paragraph("Fichier PDF Généré", styles["Heading1"]))

In [ ]:
flowables.append(Paragraph("Sébastien CHAZALLET", styles["Normal"]))

In [ ]:
flowables.append(Paragraph("http://www.inspyration.com", styles["Code"]))

In [ ]:
content = """Ce document est généré par le script 02_flux.py.
Ce script est livré avec le présent ouvrage.
Vous pouvez le modifier à souhait pour faire vos propres expériences"""

In [ ]:
flowables.append(Paragraph(content, styles["Normal"]))

In [ ]:
flowables.append(Spacer (0, 0.2*cm))

In [ ]:
from PIL import Image

In [ ]:
width, height = Image.open("image.jpg").size

In [ ]:
flowables.append(Image('image.jpg', height = 1.74 * cm, width = 1.41 * cm))

In [ ]:
pdf = SimpleDocTemplate('flow.pdf', pagesize = A4, title = 'Premier test', author = 'SCH')
# pdf.build(flowables)
# pdf.build(flowables[:])

In [ ]:
print(flowables)

Graphiques
--

In [ ]:
drawing = Drawing(10 * cm, 5 * cm)

In [ ]:
lc = HorizontalLineChart()
lc.data = [
	(0, 1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1000),
	(0, 64, 128, 192, 256, 320, 384, 448, 512, 576, 640, 704),
]

In [ ]:
legend = ["Jan", "Fev", "Mar", "Avr", "Mai", "Jun", "Jul", "Aou", "Sep", "Oct", "Nov", "Dec"]

In [ ]:
lc.categoryAxis.categoryNames = legend
lc.valueAxis.valueMin = 0
lc.valueAxis.valueMax = 1000
lc.valueAxis.valueStep = 200
lc.lines[0].strokeWidth = 2
lc.lines[1].strokeWidth = 1.5

In [ ]:
drawing.add(lc)

In [ ]:
flowables.append(drawing)

In [ ]:
pdf.build(flowables[:])

In [ ]:
data = []
line = []
line.append( Paragraph ("Technologie", styles["Normal"]) )
line.append( Paragraph ("Logiciel", styles["Normal"]) )
line.append( Paragraph ("Alternatives", styles["Normal"]) )
data.append(line)
line = []
line.append( Paragraph ("Système d'exploitation", styles["Normal"]) )
line.append( Paragraph ("Debian", styles["Normal"]) )
line.append( Paragraph ("Ubuntu, Fedora", styles["Normal"]) )
data.append(line)
line = []
line.append( Paragraph ("Serveur d'annuaires", styles["Normal"]) )
line.append( Paragraph ("openLDAP", styles["Normal"]) )
line.append( Paragraph ("lemonLDAP", styles["Normal"]) )
data.append(line)
line = []
line.append( Paragraph ("Serveur web", styles["Normal"]) )
line.append( Paragraph ("Apache2", styles["Normal"]) )
line.append( Paragraph ("LightHttpd", styles["Normal"]) )
data.append(line)

Gestion des sauts de page
--

In [ ]:
# You should execute this line to install lorem
import subprocess
print(subprocess.getstatusoutput("pip install lorem"))

In [ ]:
import lorem

In [ ]:
flowables.append(Paragraph(lorem.paragraph(), styles["Normal"]))

In [ ]:
flowables.append(Paragraph(lorem.paragraph(), styles["mon_style"]))

In [ ]:
for _ in range(16):
    flowables.append(Paragraph(lorem.paragraph(), styles["Just"]))

In [ ]:
for _ in range(2):
    flowables.append(Spacer (0, 0.2*cm))
    flowables.append(Paragraph(lorem.paragraph(), styles["Just"]))

In [ ]:
flowables.append(PageBreak())

In [ ]:
flowables.append(Table(data, colWidths=[5*cm, 5*cm, 8*cm], style=style_tableau1))

In [ ]:
pdf.build(flowables[:])

Utilisation de templates
--

In [ ]:
from reportlab.platypus.doctemplate import PageTemplate, BaseDocTemplate
from reportlab.platypus.frames import Frame
from reportlab.lib.units import cm
from reportlab.pdfgen.canvas import Canvas

In [ ]:
class FlowTemplate (PageTemplate):
    """Template for a pdf with datas in a flow."""

    def __init__ (self, parent):
        """Initialization of Template : llc = lower left corner"""

        self.parent = parent
        self.largeur = self.parent.pagesize[0]
        self.hauteur = self.parent.pagesize[1]
        self.marginx, self.marginy = 0.7 * cm, 1.4 * cm
        self.llcx = self.largeur - self.marginx
        self.llcy = 1.0 * cm
        self.page = 0
        content = Frame(self.marginx, self.marginy, self.largeur - 2 * self.marginx, self.hauteur - 2 * self.marginy)
        PageTemplate.__init__ (self, "Content", [content])

    def beforeDrawPage (self, canvas, doc):
        """before Drawing Page, we draw elements of the template"""

        canvas.saveState ()
        try:
            self.drawTemplate(canvas, doc)
        finally:
            canvas.restoreState()

    def drawTemplate(self, canvas, doc):
        """Can be overridden"""

        self.page += 1
        # Dessin d'un carré noir 
        canvas.setFillColorCMYK( 0, 0, 0, 1 )
        # canvas.setStrokeColorCMYK( 0, 0, 0, 1 )
        canvas.rect(self.llcx, self.llcy, 0.4 * cm, 0.4 * cm, stroke=0, fill=1 )
        # Ajout du numéro de la page dans le carré noir.
        canvas.setFont ('Helvetica', 8)
        canvas.setFillColorCMYK( 1, 0, 0, 0 )
        canvas.drawCentredString(self.llcx + 0.2 * cm, self.llcy + 0.1 * cm, f"{self.page}")

In [ ]:
len(flowables)

In [ ]:
doc = BaseDocTemplate(filename="flow_template_1.pdf")
doc.addPageTemplates(FlowTemplate(doc))
doc.build(flowables[:], canvasmaker=Canvas)

In [ ]:
flowables = flowables[:]
for _ in range(4):
    flowables.extend(flowables[:])

doc = BaseDocTemplate(filename="flow_template_1.pdf")
doc.addPageTemplates(FlowTemplate(doc))
doc.build(flowables[:] * 5, canvasmaker=Canvas)

In [ ]:
from reportlab.platypus import KeepTogether
altered_flowables = []
for flowable in flowables:
    if isinstance(flowable, Paragraph):
        altered_flowables.append(KeepTogether(flowable))
    else:
        altered_flowables.append(flowable)

In [ ]:
doc = BaseDocTemplate(filename="flow_template_2.pdf")
doc.addPageTemplates(FlowTemplate(doc))
doc.build(altered_flowables, canvasmaker=Canvas)

---

Retour sur les Canvas
==

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-


"""
Exemple de création d'une plaquette PDF.
"""


__author__ = "Sébastien CHAZALLET"
__copyright__ = "Copyright 2012"
__credits__ = ["Sébastien CHAZALLET", "InsPyration.org", "Éditions ENI"]
__license__ = "GPL"
__version__ = "1.0"
__maintainer__ = "Sébastien CHAZALLET"
__email__ = "sebastien.chazallet@laposte.net"
__status__ = "Production"


from reportlab.platypus.doctemplate import PageTemplate
from reportlab.platypus.doctemplate import BaseDocTemplate
from reportlab.lib import colors


class PlaquetteTemplate (PageTemplate):
    """Modèle de Pages PDF pour une plaquette commerciale"""

    def __init__ (self, context):
        self.context = context
        self.largeur = self.context.document.pagesize[0]
        self.hauteur = self.context.document.pagesize[1]
        self.zone1 = Frame(0.7*cm, 13*cm, self.largeur - 0.7*cm, 10*cm)
        self.zone2 = Frame(0.7*cm, 9*cm, self.largeur - 0.7*cm, 6*cm)
        self.zone3 = Frame(0.7*cm, 5*cm, self.largeur - 0.7*cm, 2*cm)
        PageTemplate.__init__ (self, id="Tiers", frames=[self.zone1, self.zone2, self.zone3], pagesize=A4)

    def beforeDrawPage (self, canvas, doc):
        canvas.saveState ()
        try:
            self.zone1.addFromList( self.context.flowables_zone1 , canvas)
            self.zone2.addFromList( self.context.flowables_zone2 , canvas)
            self.zone3.addFromList( self.context.flowables_zone3 , canvas)
        finally:
            canvas.restoreState ()

    def afterDrawPage (self, canvas, doc):
        canvas.saveState ()
        try:
            canvas.setFillColorRGB(*colors.mediumaquamarine.rgba())
            canvas.setStrokeColorRGB(*colors.midnightblue.rgba())
            canvas.rect(0.7*cm, self.hauteur - 2*0.7*cm, 0.7*cm, 0.7*cm, fill=1)
        finally:
            canvas.restoreState ()

class PlaquettePDF:

    def __init__ (self, context):
        self.context = context
        self.built = 0
        self.objects = [Spacer (0, 0.5*cm)]
        self.styles = getSampleStyleSheet() 
        self.flowables_zone1=[
            Paragraph("ZONE 1", self.styles['Normal']),
            Spacer (0, 0.5*cm),
            Paragraph(lorem.paragraph(), styles["mon_style"]),
            Spacer (0, 0.5*cm),
            Paragraph(lorem.paragraph(), styles["mon_style"]),
            Spacer (0, 0.5*cm),
            Paragraph(lorem.paragraph(), styles["mon_style"]),
            Spacer (0, 0.5*cm),
            Paragraph(lorem.paragraph(), styles["mon_style"]),
            Spacer (0, 0.5*cm),
            Paragraph(lorem.paragraph(), styles["mon_style"]),
            Spacer (0, 0.5*cm),
            Paragraph(lorem.paragraph(), styles["mon_style"]),
            Spacer (0, 0.5*cm),
            Paragraph(lorem.paragraph(), styles["mon_style"]),
        ]
        self.flowables_zone2=[
            Paragraph("ZONE 2", self.styles['Normal']),
            Spacer (0, 0.5*cm),
            Paragraph("xXx", styles["mon_style"]),
            Spacer (0, 0.5*cm),
            Paragraph("xXx", styles["mon_style"]),
            Spacer (0, 0.5*cm),
            Paragraph("xXx", styles["mon_style"]),
            Spacer (0, 0.5*cm),
            Paragraph("xXx", styles["mon_style"]),
            Spacer (0, 0.5*cm),
            Paragraph("xXx", styles["mon_style"]),
            Spacer (0, 0.5*cm),
            Paragraph("xXx", styles["mon_style"]),
            Spacer (0, 0.5*cm),
            Paragraph("xXx", styles["mon_style"]),
            Spacer (0, 0.5*cm),
            Paragraph("xXx", styles["mon_style"]),
        ]
        self.flowables_zone3=[
            Paragraph("ZONE 3", self.styles['Normal']),
            Paragraph("-+-+-+-+-+-+-+-+-", styles["mon_style"]),
            Paragraph("-+-+-+-+-+-+-+-+-", styles["mon_style"]),
            Paragraph("-+-+-+-+-+-+-+-+-", styles["mon_style"]),
            Paragraph("-+-+-+-+-+-+-+-+-", styles["mon_style"]),
            Paragraph("-+-+-+-+-+-+-+-+-", styles["mon_style"]),
            Paragraph("-+-+-+-+-+-+-+-+-", styles["mon_style"]),
            Paragraph("-+-+-+-+-+-+-+-+-", styles["mon_style"]),
        ]
        self.document = BaseDocTemplate ("plaquette.pdf", leftMargin=0.7*cm, rightMargin=0.7*cm, topMargin=0.7*cm, bottomMargin=0.7*cm, pagesize=A4)
        self.document.addPageTemplates ( PlaquetteTemplate (self))
        self.document.build (self.objects)
        self.built = 1

PlaquettePDF('')